In [5]:

# imports
from dataclasses import dataclass, field
from bs4 import BeautifulSoup
import requests
import csv
from datetime import date
from datetime import datetime
import json
from xmlrpc.client import DateTime

# addtional functions

# function to send a get request to a url and return the response
def getResponse(url):
    return requests.get(url)

# function to extract the text from a get request
def getHTML(response):
    return response.text

#function to format dates
def date_formatter(date_text, year):
    date_index = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
                    "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}
    day = date_text[-2:].replace(" ","0")
    month = date_index[date_text[:3]]
    formatted = f"{year}-{month}-{day}"
    return formatted
    
# function to format dates from user input
def get_date(prompt):
    is_valid = False
    while not is_valid:
        try:
            try_date = input(prompt)
            date_list = [int(x) for x in try_date.split("-")]
            y, m, d, = date_list[0], date_list[1], date_list[2]
            my_date = date(y, m, d)
            is_valid = True
        except:
            print('Invalid date, try again.')
    return my_date

In [ ]:
########################## test webscraping ######################################

# create list of years to add to url (each year has different page)
yearList = ["2020", "2021", "", "2023", "2024"]

# create empty holiday dict
holidaysDict = {"holidays": []}

# loop through pages of holiday tables to get holiday values
for year in yearList:
    url = f"https://www.timeanddate.com/holidays/us/{year}"
    res = getResponse(url)
    soup = BeautifulSoup(getHTML(res))
    table = soup.find("table", attrs = {"id": "holidays-table"}).find("tbody")
    # loop through table rows to get names and dates
    for i in range(1, len(table)):
        table_row = table.find("tr", attrs = {"id": f"tr{i}"})
        # use if/else to skip rows that are not formatted wth id : tr{number}
        if table_row is None:
            table_row = table.find("tr", attrs = {"id": f"tr1"})
        else:
            if year == "":
                holiday_date = date_formatter(table_row.find("th", attrs = {"class": "nw"}).text, "2020")
                holiday_name = table_row.find("a").text
                holidaysDict["holidays"].append({"name": holiday_name, "date": holiday_date})
            else:
                holiday_date = date_formatter(table_row.find("th", attrs = {"class": "nw"}).text, year)
                holiday_name = table_row.find("a").text
                holidaysDict["holidays"].append({"name": holiday_name, "date": holiday_date})

# load initial holiday list from file and add to temp list
with open("initial_holidays.json") as initHolidays:
    initialHolidaysList = json.load(initHolidays)
temp = []
for holiday in initialHolidaysList["holidays"]:
    temp.append((holiday["name"], holiday["date"]))

# remove duplicate values from dict list
res = []
for holiday in holidaysDict["holidays"]:
    result = (holiday["name"], holiday["date"])
    if result not in temp:
        temp.append(result)
        res.append(holiday)
print(f'len temp: {len(temp)}')
print(f'len res: {len(res)}')

# clean up date values in 2022 years

res.pop(1074)
innerHolTestList = []
for index, value in enumerate(res):
    print(index)
    date_list = [int(x) for x in value['date'].split("-")]
    y, m, d, = date_list[0], date_list[1], date_list[2]
    my_date = date(y, m, d)
    name = value['name'].strip().upper()
    newHoliday = Holiday(name, my_date)
            # Make sure holidayObj is an Holiday Object by checking the type
    if isinstance(newHoliday, object):
        # Use innerHolidays.append(holidayObj) to add holiday
        innerHolTestList.append(newHoliday)
# def scrapeHolidays():
#     # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
#     # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
#     # Check to see if name and date of holiday is in innerHolidays array
#     # Add non-duplicates to innerHolidays
#     # Handle any exceptions. 


In [ ]:
res[1100]

In [ ]:
# load initial holiday list from file and add to temp list
with open("initial_holidays.json") as initHolidays:
    initialHolidaysList = json.load(initHolidays)
temp = []
for holiday in initialHolidaysList["holidays"]:
    temp.append((holiday["name"], holiday["date"]))
temp

In [ ]:
with open("initial_holidays.json") as initHolidays:
    initialHolidaysList = json.load(initHolidays)
initialHolidaysList

In [46]:
###################### test functions ##############################

@dataclass
class Holiday:
    name: str
    date: date #= field(metadata={"units":"U.S. dollars"})
    
    def __str__(self):
        return f"Name: {self.name}, Date: {self.date}"
# test Holiday class
# holiday1 = Holiday("Valentine's Day", "2022-02-14")
# print(holiday1)

class HolidayList:
    def __init__(self):
       self.innerHolidays = []
       self.filelocation = "initial_holidays.json"
   
    def addHoliday(self):
    # function to get new holiday from user and append to innerHolidays list
        while True:
            name_input = input("Please enter new holiday name: ")
            name = name_input.strip().upper()
            date = get_date("Enter date in the format [YYYY-MM-DD], i.e. [2021-06-02]: ")
            new_holiday = Holiday(name, date)
             # Make sure holidayObj is an Holiday Object by checking the type
            if isinstance(new_holiday, object):
                # Use innerHolidays.append(holidayObj) to add holiday
                if new_holiday in self.innerHolidays:
                    print('Already in holiday list.')
                else:
                    self.innerHolidays.append(new_holiday)
                    print(f"You added {new_holiday} successfully.")
                    break
            else:
                print("Please enter a valid name and date.")
        # print to the user that you added a holiday

    def findHoliday(self, HolidayName):
        # function to find and print out holiday in innerHoliday list
        # search for Holiday Name
        for index, holiday in enumerate(self.innerHolidays):
            # [test] print(index, holiday)
            if holiday == HolidayName:
                # [test] print(f"findHoliday index, type(index) result: {index, type(index)}")
                return index
        # [test] print(f"Holiday not found. Please try again.")
        return None

        # Find Holiday in innerHolidays
        # Return Holiday

    def removeHoliday(self):
        # Find Holiday in innerHolidays by searching the name and date combination.
        while True:
            name_input = input("Please enter holiday name to remove: ")
            name = name_input.strip().upper()
            date = get_date("Enter date in the format [YYYY-MM-DD], i.e. [2021-06-02]: ")
            searchHoliday = Holidays.findHoliday(HolidayName = Holiday(name, date))
            # [test] print(f"removeHoliday: searchHoliday result: {searchHoliday}")
            # check findHoliday result
            # remove the Holiday from innerHolidays
            if searchHoliday != None:
                self.innerHolidays.pop(searchHoliday)
                # inform user you deleted the holiday
                print(f"{name}: {date} removed successfully.")
                break
            else:
                print(f"Error: {name}: {date} not found.")
        return # [test] print(f"test result: {self.innerHolidays}")
        
    def readjson(self, filelocation):
        # Read in things from json file location
        with open(filelocation) as initHolidays:
            initialHolidaysList = json.load(initHolidays)
        # [test] return initialHolidaysList
        # [test] test_list = []
        # Use addHoliday function to add holidays to inner list.
        for index, value  in enumerate(initialHolidaysList["holidays"]):
            date_list = [int(x) for x in value['date'].split("-")]
            y, m, d, = date_list[0], date_list[1], date_list[2]
            my_date = date(y, m, d)
            name = value['name'].strip().upper()
            newHoliday = Holiday(name, my_date)
                    # Make sure holidayObj is an Holiday Object by checking the type
            if isinstance(newHoliday, object):
                # Use innerHolidays.append(holidayObj) to add holiday
                self.innerHolidays.append(newHoliday)
        return print(f"test print {self.innerHolidays}")

    def save_to_json(self):
        holiList = [holi.__dict__ for holi in Holidays.innerHolidays]
        finalHoliList = []
        for hol in holiList:
            tempDict = {}
            tempDict["name"] = hol["name"]
            tempDict["date"] = (hol["date"]).strftime("%Y/%m/%d")
            finalHoliList.append(tempDict)
        with open("holidayList.json", "w") as f:
            json.dump(finalHoliList, f)

    def scrapeHolidays(self):
        yearList = ["2020", "2021", "", "2023", "2024"]

        # create empty holiday dict
        holidaysDict = {"holidays": []}

        # loop through pages of holiday tables to get holiday values
        for year in yearList:
            url = f"https://www.timeanddate.com/holidays/us/{year}"
            res = getResponse(url)
            soup = BeautifulSoup(getHTML(res))
            table = soup.find("table", attrs = {"id": "holidays-table"}).find("tbody")
            # loop through table rows to get names and dates
            for i in range(1, len(table)):
                table_row = table.find("tr", attrs = {"id": f"tr{i}"})
                # use if/else to skip rows that are not formatted wth id : tr{number}
                if table_row is None:
                    table_row = table.find("tr", attrs = {"id": f"tr1"})
                else:
                    # add year manually to 2022 as it's missing from the url
                    if year == "":
                        holiday_date = date_formatter(table_row.find("th", attrs = {"class": "nw"}).text, "2022")
                        holiday_name = table_row.find("a").text
                        holidaysDict["holidays"].append({"name": holiday_name, "date": holiday_date})
                    else:
                        holiday_date = date_formatter(table_row.find("th", attrs = {"class": "nw"}).text, year)
                        holiday_name = table_row.find("a").text
                        holidaysDict["holidays"].append({"name": holiday_name, "date": holiday_date})

        # load initial holiday list from file and add to temp list
        with open("initial_holidays.json") as initHolidays:
            initialHolidaysList = json.load(initHolidays)
        temp = []
        for holiday in initialHolidaysList["holidays"]:
            temp.append((holiday["name"], holiday["date"]))

        # remove duplicate values from dict list and Check to see if name and date of holiday is in innerHolidays array
        res = []
        for holiday in holidaysDict["holidays"]:
            result = (holiday["name"], holiday["date"])
            if result not in temp:
                temp.append(result)
                res.append(holiday)
        print(f'len temp: {len(temp)}')
        print(f'len res: {len(res)}')

        # [test] innerHolTestList = []
        for index, value in enumerate(res):
            # [test] print(index)
            date_list = [int(x) for x in value['date'].split("-")]
            y, m, d, = date_list[0], date_list[1], date_list[2]
            my_date = date(y, m, d)
            name = value['name'].strip().upper()
            newHoliday = Holiday(name, my_date)
                    # Make sure holidayObj is an Holiday Object by checking the type
            if isinstance(newHoliday, object):
                # Use innerHolidays.append(holidayObj) to add holiday
                self.innerHolidays.append(newHoliday)    

    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        numHolidays = len(self.innerHolidays)
        return print(f"There are {numHolidays} in the system currently")

    def filterHolidaysByWeek(self, yearNum, weekNum):
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        filteredHolidaysWeek = list(filter(lambda x: x.date.isocalendar().week == weekNum, self.innerHolidays))
        filteredHolidaysYearWeek = list(filter(lambda x: x.date.isocalendar().year == yearNum, filteredHolidaysWeek))
        return filteredHolidaysYearWeek


In [7]:
########################### test main ##############################
# initialize HolidayList
Holidays = HolidayList()
Holidays.readjson(filelocation="initial_holidays.json")
#Holidays.scrapeHolidays()
#Holidays.numHolidays()
Holidays.addHoliday()
Holidays.removeHoliday()
Holidays.save_to_json()

test print [Holiday(name='MARGARET THATCHER DAY', date=datetime.date(2021, 1, 10)), Holiday(name='WORLD SKETCHNOTE DAY', date=datetime.date(2021, 1, 11)), Holiday(name='ZANZIBAR REVOLUTION DAY', date=datetime.date(2021, 1, 12)), Holiday(name='NATIONAL RUBBER DUCKY DAY', date=datetime.date(2021, 1, 13)), Holiday(name='TAMIL THAI PONGDAL DAY', date=datetime.date(2021, 1, 14)), Holiday(name='NATIONAL BAGEL DAY', date=datetime.date(2021, 1, 15)), Holiday(name='SIGNING OF THE PEACE ACCORDS', date=datetime.date(2021, 1, 16))]
len temp: 2301
len res: 2294
There are 2301 in the system currently
You added Name: POOP DAY, Date: 2022-08-29 successfully.
POOP DAY: 2022-08-29 removed successfully.


In [22]:
print(Holidays.innerHolidays[0].date) #> date(2021, 1, 11)
print(Holidays.innerHolidays[0].date.isocalendar().week)
print(Holidays.innerHolidays[0].date.strftime("%U"))
print(Holidays.innerHolidays[0].date.strftime("%V"))

2021-01-10
1
02
01


In [47]:
Holidays.filterHolidaysByWeek(2021, 2)

AttributeError: 'HolidayList' object has no attribute 'filterHolidaysByWeek'

In [38]:
Holidays.innerHolidays[1500]

Holiday(name='MEMORIAL DAY', date=datetime.date(2023, 5, 29))